In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem import PandasTools
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn import model_selection
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [2]:
def get_fps(mol):
    # this calculates all RDKit descriptors (clogP, PSA, etc etc)
    calc=MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    ds = np.asarray(calc.CalcDescriptors(mol))
    
    # EState fingerprints
    arr=Fingerprinter.FingerprintMol(mol)[0]
    
    # Morgan fingerprints
    #fps=AllChem.GetMorganFingerprintAsBitVect(mol,3,nBits=1024)
    #arr=np.zeros((1,))
    #DataStructs.ConvertToNumpyArray(fps, arr)
    
    return np.append(arr,ds)

In [3]:
# load data from CSV. Keep only smiles, compound name and pIC50 columns
# the data set is here: https://github.com/deepchem/deepchem/blob/master/datasets/desc_canvas_aug30.csv
df=pd.read_csv('desc_canvas_aug30.csv',usecols=[0,1,4])

In [4]:
# randomly shuffle the data
df = shuffle(df)

In [5]:
df.head()

,mol,CID,pIC50
956,Fc1cc(cc(F)c1)CC(NC(=O)C)C(O)C[NH2+]C1(CC1)c1cc(ccc1)CC(C)(C)C,BACE_982,6.467246
257,Fc1cc(cc(F)c1)CC(NC(=O)C(N1CCC(NC(=O)C)(C(CC)C)C1=O)CCc1ccccc1)C(O)C1[NH2+]CC(OCc2cccnc2)C1,BACE_283,8.853872
552,Fc1ncccc1-c1cc(ccc1)C1(N=C(N2C1=NOCC2)N)c1ccc(OC(F)(F)F)cc1,BACE_578,7.522879
1142,FC(F)(F)Oc1ccc(cc1)C1([NH+]=C(N2C1=NCCC2)N)c1cc(ccc1)CCC,BACE_1168,5.844664
998,Clc1cc2CC(N=C(NC(Cc3ccccc3)C=3NC(=O)C(=CN=3)C#N)c2cc1)(C)C,BACE_1024,6.327902


In [ ]:
# add RDKit molecule column to the dataframe
PandasTools.AddMoleculeColumnToFrame(df,'mol','Molecule')

In [ ]:
# calculate molecular fingerprints and descriptors and add to the dataframe
df['Descriptors']=df['Molecule'].apply(get_fps)

In [ ]:
# tag molecules with pIC50>6 as active (Active = 1)
df['Active']=np.where(df['pIC50']>6, 1, 0)

In [ ]:
df.head()

In [ ]:
# convert to numpy arrays
X = np.array(list(df['Descriptors']))
y = df['Active'].values
 
# divide into train and test set
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

In [ ]:
# create a random forest model and fit the data
rf = RandomForestClassifier(max_features='auto')
rf.fit(X_train, y_train)

In [ ]:
# make predictions on the test set
y_pred = rf.predict(X_test)

In [ ]:
# get the ROC statistics
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

In [ ]:
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b',
label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()